# Efficient	String	Matching : Aho Corasick algorithm
## Madalina Ciortan

__Implementation details:__   
Python 3.5  
No libraries



Global variables used: 

__g__ : this dictionary represents go to function and holds as keys tuples (state, weight) and the value is the next state  
The choice of a tuple facilitates the fast retrieval of values for the following state based on the composite key of state and existing weight, the same effect could have been obtained by concatenating as a string the 2 before mentioned properties.   
__output__ : this dictionary maps states to a list of unique keywords found  
__failure__ : this dictionary maps each state with the next state  
__caseSensitive__ : when this parameter is set to false, we match strings with letter case difference  
__fail__ = None : global constant with a meaningful name which is the output of the graph f when the weight(letter) is not part of the graph  
__newstate__ : global variable used to propagate the state index cross kewords iterations


For a clear example of the structure maintained by these variables, check the print statements after the creation of the state machine at the end of this file.
    

In [4]:
g = {} 
output = {} 
failure = {} 
caseSensitive = False 


fail = None 
newstate = 0 

The following method resets global parametrs to default values:

In [7]:
def initializeGlobalParameters():
    ''''
    This method makes sure that all global parameters are properly reset
    '''
    global g
    global output
    global failure
    global newstate
    g = {}
    output = {}
    failure = {}
    newstate = 0

The state machine is created in 2 phases: the first one creates the __go to__  and the initial version of __output__ functions while the second step creates the __failure function__ and updates __output__.   
Before running these 2 phases we want to make sure that the global parameters are set to default values.

In [10]:
def createStateMachine(keywords):
    ''''
    This method initializes global parameters, creates the go to, failure and output functions
    '''
    initializeGlobalParameters()
    createGoToFunction(keywords)
    createFailueFunction()

In [12]:
def createGoToFunction(keywords):
    '''

    :param keywords: list of strings based on which we create the automata
    :return: go to function
    '''

    newstate = 0 # start with state 0
    for keyword in keywords:
        processKeyword(keyword)

    # For all letters which are not mapped to the 0 root state, create references to state 0
    # For efficiency concatenate all keywords and remove duplicated letters with set
    allkeys = ''.join(set(''.join(keywords)))
    for i in range(len(allkeys)):
        if g.get((0, allkeys[i]), fail) == fail:
            g[(0, allkeys[i])] = 0
    return g


def processKeyword(keyword):
    '''
    For each letter in keyword, if it is already present in our graph g at current state,
    update current state to the value of the next state
    otherwise add a new node to the graph mapping the current state + 1 with the current letter
    :param keyword:
    :return:
    '''
    global newstate
    state = 0
    j = 0
    while g.get((state, keyword[j]), fail) != fail: #state already defined
        state = g[(state, keyword[j])] #we are sure the key exists
        j = j + 1

    for p in range(j, len(keyword)) :
        newstate = newstate + 1
        g[(state, keyword[p])] = newstate
        state = newstate

    updateOutput(state, keyword)


def updateOutput(state, keyword) :
    '''
    This method updates output dictionary with value keyword at key state.
    Because we can find multiple keywords at the same index, output maintains a list of
    keywords.
    For this reason, if input keyword is a string and the key doest't exist, I create an array
    with given value, otherwise I add keyword to existing array.
    This method is called by processKeyword and createFailure function and in the latter case
    it may pass a list of values (the payload of another output node, for this reason
    I check the instance type and do an array extends.
    '''
    if state in output.keys():
        #this scenario will be triggered by updates from create failure
        if isinstance(keyword, list):
            output[state].extend(keyword)
        else:
            output[state].append(keyword)
    else:
        output[state] = [keyword]


def createFailueFunction():
    '''
    This method iterates through all letters of keywords payload (set method guarantees
    distinct output) and uses a queue to manange states.
    '''
    queue = []
    allkeys = ''.join(set(''.join(keywords)))
    for i in range(len(allkeys)):
        s = g.get((0, allkeys[i]), fail)
        if  s != 0:
            queue.append(s)
            failure[s] = 0

    while len(queue) != 0 :
        r = queue[0]
        queue.remove(queue[0])

        for i in range(len(allkeys)):
            s = g.get((r, allkeys[i]), fail)
            if s!= fail :
                queue.append(s)
                state = failure[r]
                while g.get((state, allkeys[i]), fail) == fail :
                    state = failure[state]
                failure[s] = g[(state, allkeys[i])]
                if failure[s] in output.keys():
                    newOutputValue = output[failure[s]]
                    updateOutput(s, newOutputValue)

The following method takes as input the string to be parsed and a file name where the result will be written to.  
The content of this output file will be _index_: _keywords_  ( see output file on github)  
e.g: Index: 19 ['the']   
Index: 52 ['pattern']   
Index: 77 ['tree']   

The program will also print to the console this results together with the string up to the point where the match was found:

In [14]:
def findInString(inputString, outputFileName):
    '''
    Find all occurences of keywords in given inputString
    This method has to be called after createStateMachine
    '''
    state = 0
    outputFile = open( outputFileName, 'w')
    for i in range(len(inputString)) :
        #failure function doesn't have a value for state 0 therefor make sure the
        #this loop doesn't execute the very first time
        while g.get((state, inputString[i]), fail) == fail and state != 0:
            state = failure[state]
        #update state only when the tuple state, inputString[i] exists in go to graph, otherwise keep existing state
        state = g.get((state, inputString[i]), state)
        if output.get(state, None ) != None :
            print('\nIndex : %s , Output keywords %s \nActual text : %s' % (i, output[state], inputString[:i + 1]))
            outputFile.write('Index: %s %s \n' % (i, output[state]))


The following method reads the input from given file and returns it as string:

In [16]:
def readTextFromFile(filename):
    '''
    Return string payload of input filename.

    There might be differences in the output index results due to extra characters passed
    along during the copy paste to the input file.
    '''
    with open(filename, 'r') as myfile:
        text = myfile.read()
        if not caseSensitive :
            return text.lower()
        return text

Load keywords and text from file. (see input file on github where the given text was copied)  
If caseSensistive parameter is provided, convert everything to lowercase.

In [19]:
keywords = ['pattern',	'tree',	'state','prove','the','it']
if caseSensitive :
    keywords = [x.lower() for x in keywords]

text = readTextFromFile('input')

Create state machine and inspect goto/failure/output functions

In [21]:
createStateMachine(keywords)
print( '>Go To Graph : %s \n>Ouput function : %s \n>Failure function : %s' % (g, output, failure))

>Go To Graph : {(0, 'n'): 0, (6, 'n'): 7, (18, 'v'): 19, (21, 'e'): 22, (17, 'o'): 18, (3, 't'): 4, (0, 'p'): 1, (10, 'e'): 11, (0, 'r'): 0, (0, 's'): 12, (1, 'a'): 2, (8, 'r'): 9, (0, 't'): 8, (0, 'v'): 0, (0, 'e'): 0, (5, 'r'): 6, (0, 'i'): 23, (13, 'a'): 14, (15, 'e'): 16, (23, 't'): 24, (8, 'h'): 21, (2, 't'): 3, (1, 'r'): 17, (0, 'a'): 0, (0, 'h'): 0, (9, 'e'): 10, (4, 'e'): 5, (12, 't'): 13, (0, 'o'): 0, (14, 't'): 15, (19, 'e'): 20} 
>Ouput function : {16: ['state'], 20: ['prove'], 22: ['the'], 7: ['pattern'], 24: ['it'], 11: ['tree']} 
>Failure function : {1: 0, 2: 0, 3: 8, 4: 8, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 8, 14: 0, 15: 8, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 8}


Find indexes in input text:

In [23]:
findInString(text, 'output')


Index : 19 , Output keywords ['the'] 
Actual text : as	discussed	in	 the

Index : 52 , Output keywords ['pattern'] 
Actual text : as	discussed	in	 the	session	on	combinatorial	pattern

Index : 77 , Output keywords ['tree'] 
Actual text : as	discussed	in	 the	session	on	combinatorial	pattern	matching,	keywords	 tree

Index : 141 , Output keywords ['pattern'] 
Actual text : as	discussed	in	 the	session	on	combinatorial	pattern	matching,	keywords	 trees
provide	an	efficient	solution	to	search	for	multiple	k pattern

Index : 169 , Output keywords ['the'] 
Actual text : as	discussed	in	 the	session	on	combinatorial	pattern	matching,	keywords	 trees
provide	an	efficient	solution	to	search	for	multiple	k patterns	in	a	text	of	length	n.
the

Index : 177 , Output keywords ['it'] 
Actual text : as	discussed	in	 the	session	on	combinatorial	pattern	matching,	keywords	 trees
provide	an	efficient	solution	to	search	for	multiple	k patterns	in	a	text	of	length	n.
the	algorit

Index : 200 , Output ke

Complexity  

As proven in the article, the above implemented algorithms run in linear time, findInString makes fewer than 2n state transitions in processing a text string of length n.
